In [30]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [31]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
file_path = Path("lending_data.csv")
lending_data_df = pd.read_csv(file_path)

# Review the DataFrame
lending_data_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [32]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_data_df['loan_status']

# Separate the X variable, the features
X = lending_data_df.drop(columns='loan_status')

In [33]:
# Review the y variable Series
print(y.shape)
print(y[:10])

(77536,)
0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: loan_status, dtype: int64


In [34]:
# Review the X variable DataFrame
X.head(10)

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000
5,10100.0,7.438,50600,0.407115,4,1,20600
6,10300.0,7.490,51100,0.412916,4,1,21100
7,8800.0,6.857,45100,0.334812,3,0,15100
8,9300.0,7.096,47400,0.367089,3,0,17400
9,9700.0,7.248,48800,0.385246,4,0,18800


In [35]:
X.shape

(77536, 7)

### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [36]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [51]:
# train_test_split module imported at top of document

# Split the data using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)


---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [38]:
# LogisticRegression module from SKLearn imported at top of document

# Instantiate the Logistic Regression model
classifier = LogisticRegression(solver='lbfgs',
                                random_state=1)

# Fit the model using training data
classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [39]:
# Print model scores
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}") 

Training Data Score: 0.9914878250103177
Testing Data Score: 0.9924164259182832


In [40]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)
results = pd.DataFrame({'Prediction': predictions, 'Actual': y_test}).reset_index(drop=True)
results.head(10)

,Prediction,Actual
0,0,0
1,0,1
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [41]:
# Print the balanced_accuracy score of the model
testing_accuracy = balanced_accuracy_score(y_test, predictions)
print(testing_accuracy)

0.9442676901753825


In [42]:
# Generate a confusion matrix for the model
testing_matrix = confusion_matrix(y_test, predictions)
print(testing_matrix)

[[18679    80]
 [   67   558]]


In [43]:
# Print the classification report for the model
testing_report = classification_report(y_test, predictions)
print(testing_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      0.89      0.88       625

    accuracy                           0.99     19384
   macro avg       0.94      0.94      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** 

Out of all the accounts the model predicted would be high risk, 87% actually were high risk.

Out of all the accounts that actually were high risk, the model predicted this outcome for 89% of the accounts.

With a balanced accuracy score of 94%, the model does a very good job of predicting which accounts are at high risk of defaulting (since this is above 90%).

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [44]:
# RandomOverSampler module from imbalanced-learn imported at top of document

# Instantiate the random oversampler model
ros = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_over, y_over = ros.fit_resample(X_train, y_train)

In [45]:
# Count the distinct values of the resampled labels (y) data
X_over.shape

(112554, 7)

In [46]:
# Count the distinct values of the resampled features (x) data
y_over.shape

(112554,)

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [47]:
# Instantiate the Logistic Regression model
over_classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
over_classifier.fit(X_over, y_over)

# Make a prediction using the testing data
over_predictions = over_classifier.predict(X_test)
results = pd.DataFrame({'Prediction': predictions, 'Actual': y_test}).reset_index(drop=True)
results.head(10)

,Prediction,Actual
0,0,0
1,0,1
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [48]:
# Print the balanced_accuracy score of the model 
over_testing_accuracy = balanced_accuracy_score(y_test, over_predictions)
print(over_testing_accuracy)

0.9959744975744975


In [49]:
# Generate a confusion matrix for the model
over_testing_matrix = confusion_matrix(y_test, over_predictions)
print(over_testing_matrix)

[[18668    91]
 [    2   623]]


In [50]:
# Print the classification report for the model
over_testing_report = classification_report(y_test, over_predictions)
print(over_testing_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      1.00      0.93       625

    accuracy                           1.00     19384
   macro avg       0.94      1.00      0.96     19384
weighted avg       1.00      1.00      1.00     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:**

The RandomOverSampler model operates by randomly duplicating values in the minority class to better combat data bias and more equally represent all classes within the data.

In the oversampled data, out of all the accounts the model predicted would be high risk, 87% actually were high risk. This is equivalent to the model using the original dataset.

In the oversampled data, out of all the accounts that actually were high risk, the model predicted this outcome for 100% of the accounts.

With a balanced accuracy score of 99.6%, the model does an excellent job of predicting which accounts are at high risk of defaulting.